In [1]:
import seaborn as sn
import fnmatch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils import data
import numpy as np
import torchvision
from  numpy import exp,absolute
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
from sklearn.svm import SVC 
from sklearn.neural_network import MLPClassifier as mlp
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, TensorDataset, Subset
import pandas as pd
from torchmetrics.functional.classification import multiclass_confusion_matrix as mcm
from sklearn.dummy import DummyClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score,f1_score,precision_score ,recall_score 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import random
import warnings
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
import sklearn
import cv2
from PIL import Image
import warnings
from tqdm import tqdm
from collections import defaultdict
from pathlib import Path


In [2]:
#hyper params
dataset_path = './data/binary_dataset/data'

bs = 64
test_size = 0.15  # Test set size (20%)
val_size = 0.2   # Validation set size (25%)
num_epoch = 500
patience = 20
classes = ['IRRELEVANT', 'RELEVANT'] #arranged in order of their placement in the folder
num_classes = len(classes)


In [3]:
def transform_denoise(image):
    # Denoising transformation (replace with your denoising algorithm)
    # Example: Apply Gaussian blur with a kernel size of 3
    denoised_image = transforms.functional.gaussian_blur(image, kernel_size=3,sigma=1.5)
    return denoised_image

def transform_enhance(image):
    # Image enhancement transformation (replace with your enhancement algorithm)
    # Example: Apply contrast enhancement
    enhanced_image = transforms.functional.adjust_contrast(image, contrast_factor=1.5)
    return enhanced_image

In [4]:
class DatasetFromSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

In [5]:
def subsample(dataset, r=0.):
    # Get the labels for the dataset
    labels = np.asarray(dataset.dataset.imgs)[:, 1][dataset.indices]

    # Count the number of samples per class
    class_counts = defaultdict(int)
    for label in labels:
        class_counts[label] += 1

    # Calculate m
    m = min(class_counts.values())

    # Calculate the threshold for each class
    threshold = m + r * m

    # Create a new subset of the dataset
    indices = []
    new_class_counts = defaultdict(int)
    for i in range(len(dataset)):
        label = labels[i]
        if new_class_counts[label] < threshold:
            indices.append(i)
            new_class_counts[label] += 1

    return Subset(dataset, indices)

In [6]:
def get_dataset(path,val_split,test_split):
    transform_train = transforms.Compose([
                                        transforms.Resize((256, 256)),
                                        transforms.Lambda(lambda x: transform_denoise(x)),
                                        transforms.Lambda(lambda x: transform_enhance(x)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomRotation(degrees=10),
                                        transforms.RandomCrop(size=256),
                                        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                        ])
    transform_test = transforms.Compose([
                                        transforms.Resize((256, 256)),
                                        transforms.Lambda(lambda x: transform_denoise(x)),
                                        transforms.Lambda(lambda x: transform_enhance(x)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                        ])


    dataset = datasets.ImageFolder(path)    
    trainset,valset,testset = stratified(dataset, r=.0)

    trainset = DatasetFromSubset(trainset, transform=transform_train)
    valset = DatasetFromSubset(valset, transform=transform_test)
    testset = DatasetFromSubset(testset, transform=transform_test)
    
    print("train :",len(trainset),"val :",len(valset),"test :",len(testset))
    
    return trainset,valset,testset


In [7]:
def write_indices(indices, images, name):
    file_list = np.asarray(images)[indices]

    with open(Path(dataset_path) / (name + '.txt'), 'w') as f:
        f.writelines(file_list)


def stratified(dataset, r=0.):
    # Get the labels and targets from the dataset
    labels = np.asarray(dataset.imgs)[:, 1]

    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    train_val_indices, test_indices = next(stratified_split.split(np.zeros(len(labels)), labels))

    train_val_dataset = torch.utils.data.Subset(dataset, train_val_indices)
    testset = torch.utils.data.Subset(dataset, test_indices)

    # write down the splits
    imgs = [img[0] + '\n' for img in dataset.imgs]
    write_indices(train_val_indices, imgs, "train_images")
    write_indices(test_indices, imgs, "test_images")
    with open(Path(dataset_path) / "all_images.txt", "w") as f:
        f.writelines(imgs)

    # balance the train set
    train_val_dataset = subsample(train_val_dataset, r=r)
    
    # Further split the train-val dataset into train and validation sets
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=42)
    train_indices, val_indices = next(stratified_split.split(train_val_dataset, labels[train_val_indices][train_val_dataset.indices]))

    trainset = torch.utils.data.Subset(train_val_dataset, train_indices)
    valset = torch.utils.data.Subset(train_val_dataset, val_indices)

    return trainset,valset,testset

In [8]:
# trainset, valset, testset = get_dataset(Path(dataset_path) / 'data', val_size, test_size)

In [9]:
def entropy(tensor):
    probabilities = torch.softmax(tensor, dim=1)
    print(probabilities.shape)
    log_prob = torch.log2(torch.min(torch.ones_like(probabilities), probabilities + 1e-10)) / math.log2(tensor.shape[-1])
    return -torch.sum(probabilities * log_prob, dim=1)

In [10]:
def train_model(trainset,valset, model, criterion, optimizer, num_epochs, bs):
    dataloaders_strong = {
        'train': data.DataLoader(trainset, batch_size=bs, shuffle=True),
        'val': data.DataLoader(valset, batch_size=bs, shuffle=True)
    }
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, epochs=num_epoch, steps_per_epoch=len(dataloaders_strong['train']), max_lr=0.1)
    dataset_sizes_strong = {'train': len(trainset), 'val': len(valset)}
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = np.inf
    
    epochs_without_improvement = 0  # Reset the counter since there is improvement

    # Lists to store entropy values for each epoch
    entropy_values_train = []
    entropy_values_val = []
    # Lists to store loss of each epoch
    E_loss_train = []
    E_loss_val = []
    E_accuracy_train = []
    E_accuracy_val = []
    # Lists to store balanced accuracy for training and validation phases
    balanced_acc_train = []
    balanced_acc_val = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        c = 0

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            all_labels = []
            all_preds = []
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                        
            running_loss = 0.0
            running_corrects = 0
            epoch_entropy = 0.0  # Variable to store epoch entropy

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders_strong[phase], "Running " + phase):
                inputs = inputs.to(device)                
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)
                    # Calculate balanced accuracy only if all unique predicted classes are present in true labels
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore", category=UserWarning)
                        
                    all_labels.extend(labels.cpu().tolist())
                    all_preds.extend(preds.cpu().tolist())

                    # Backpropagation and optimization
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   
                        scheduler.step()               
                    
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Calculate entropy for each batch and accumulate for epoch entropy
                e = entropy(outputs)
                epoch_entropy += torch.sum(e)

            
            if phase == 'train':
                epoch_loss = running_loss / dataset_sizes_strong[phase]
                epoch_acc = running_corrects.double() / dataset_sizes_strong[phase]
                epoch_entropy /= dataset_sizes_strong[phase] # Calculate average epoch entropy
            else:
                epoch_loss = running_loss / (dataset_sizes_strong[phase]-c)
                epoch_acc = running_corrects.double() / (dataset_sizes_strong[phase]-c)
                epoch_entropy /= (dataset_sizes_strong[phase]-c) # Calculate average epoch entropy

                
                # print('Number of skipped:', c)
                # print()
            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('{} Epoch Entropy: {:.4f}'.format(phase, epoch_entropy.item()))
            print()

            if phase == 'train':
                entropy_values_train.append(epoch_entropy.cpu().item())
                E_loss_train.append(epoch_loss)
                E_accuracy_train.append(epoch_acc.cpu().item())
            else:
                entropy_values_val.append(epoch_entropy.cpu().item())
                E_loss_val.append(epoch_loss)
                E_accuracy_val.append(epoch_acc.cpu().item())
                      
            # Deep copy the model if the validation accuracy improves
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch_val = epoch
                best_acc = epoch_acc
                
                
            # Early stopping criteria
            if phase == 'val' and epoch_loss <= 1e-7:
                    print('Loss reached 0...')
                    return (
                        model,
                        entropy_values_train,
                        entropy_values_val,
                        E_loss_train,
                        E_loss_val,
                        E_accuracy_train,
                        E_accuracy_val
                    ) 
                    
            if phase == 'val' and epoch > 0:

                if epoch_loss >= best_loss:
                    epochs_without_improvement += 1
                else:
                    epochs_without_improvement = 0

                if epochs_without_improvement >= patience:
                    print('Early stopping due to no improvement in validation loss.')
                    return (
                        model,
                        entropy_values_train,
                        entropy_values_val,
                        E_loss_train,
                        E_loss_val,
                        E_accuracy_train,
                        E_accuracy_val
                    )                 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Best Val Acc. was achieved at epoch', best_epoch_val)
    # Load best model weights
    model.load_state_dict(best_model_wts)

    return (
        model,
        entropy_values_train,
        entropy_values_val,
        E_loss_train,
        E_loss_val,
        E_accuracy_train,
        E_accuracy_val
    )
  

In [11]:
def test_model(model, testset):
    running_corrects = 0
    testloader = data.DataLoader(testset, batch_size=bs, shuffle=True)
    all_labels = []
    all_preds = []
    all_confidences = []
    c = 0
    confidence_scores_distribution = []
    model.eval()
    
    for inputs, labels in tqdm(testloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Calculate confidence score for each sample in the batch
            confidence_scores = 1 - entropy(outputs)
                
            # running_corrects += torch.sum(preds == labels.data)
            all_labels.extend(labels.cpu().tolist())
            all_preds.extend(preds.cpu().tolist())
            all_confidences.extend(confidence_scores.cpu().tolist())

    return np.asarray(all_labels), np.asarray(all_preds), np.asarray(all_confidences)

In [12]:
def set_parameter_requires_grad(model, requires_grad=True):
    for param in model.parameters():
        param.requires_grad = requires_grad

In [13]:
def initialize_model(model_name, num_classes, use_pretrained=True):
    model_ft = None
    input_size = 256

    if model_name == "resnet":
        # Resnet50
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, requires_grad=True)
        model_ft.fc = nn.Linear(model_ft.fc.in_features, num_classes)
        

    elif model_name == "vgg":
        # VGG16_bn
        model_ft = models.vgg16_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, requires_grad=True)
        model_ft.classifier[6] = nn.Linear(4096, num_classes)

    elif model_name == "googlenet":
        # GoogLeNet
        model_ft = torchvision.models.googlenet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, requires_grad=True)
        model_ft.fc = nn.Linear(1024, num_classes)

    elif model_name == "densenet":
        # Densenet
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, requires_grad=True)
        model_ft.classifier = nn.Linear(1024, num_classes)

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


In [14]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda:0


In [15]:
def plot_distribution_by_class(subset):
    if isinstance(subset.subset.dataset, Subset):
        # validation and train sets
        data_ = subset.subset.dataset.dataset.dataset
        labels = np.asarray(data_.imgs)[subset.subset.dataset.dataset.indices]
        labels = labels[subset.subset.dataset.indices]
        labels = labels[subset.subset.indices, 1]
    else:
        # test set (two subsets less)
        data_ = subset.subset.dataset
        labels = np.asarray(data_.imgs)[subset.subset.indices, 1]
        
    classes = data_.classes
    labels, count = np.unique(np.asarray(labels), return_counts=True)
    labels = np.asarray([classes[int(i)] for i in labels])

    plt.figure(figsize=(10, 6))  # Set the size of the figure

    x = np.arange(len(labels))  # Generate an array of class indices
    plt.bar(x, count)
    plt.xticks(x, labels, rotation=45, ha='right')  # Set custom x-axis tick positions and labels
    plt.xlabel("Classes")
    plt.ylabel("Count")
    plt.title("Class Count")
    plt.show()

In [16]:
# Setup the loss function
criterion = nn.CrossEntropyLoss()
# Separate the train, val, and test sets
trainset,valset,testset = get_dataset(dataset_path, val_size, test_size)

train : 37640 val : 9410 test : 29724


In [ ]:
plot_distribution_by_class(trainset)
plot_distribution_by_class(valset)
plot_distribution_by_class(testset)

In [ ]:
def train_model_name(model_name):
    print(model_name)
    # Initialize the model
    model, input_size = initialize_model(model_name, num_classes, use_pretrained=True)
    
    
    # Send model to GPU, if available
    model = model.to(device)
    
    # Observe that all parameters are being optimized
    optimizer = optim.SGD(model.parameters())
    model, entropy_values_train, entropy_values_val, E_loss_train, E_loss_val, E_accuracy_train,E_accuracy_val = train_model(
        trainset,valset,model, criterion, optimizer, num_epoch, bs)

        
    # Plot entropy values
    plt.figure()
    plt.plot(entropy_values_train, label='Train')
    plt.plot(entropy_values_val, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Entropy')
    plt.title('Entropy Trend')
    plt.legend()
    plt.show()
    
    # Plot loss values
    plt.figure()
    plt.plot(E_loss_train, label='Train')
    plt.plot(E_loss_val, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Trend')
    plt.legend()
    plt.show()
    
    # Plot balanced loss values
    plt.figure()
    plt.plot(E_accuracy_train, label='Train')
    plt.plot(E_accuracy_val, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Trend')
    plt.legend()
    plt.show()

    return model

In [ ]:
model_list = []
model_list.append(train_model_name('densenet'))

In [ ]:
model_list.append(train_model_name('resnet'))

In [ ]:
model_list.append(train_model_name('googlenet'))

In [ ]:
# saving models to file
for i, model in enumerate(model_list):

    # Define the file path to save the model
    model_file1 = f'./model_state_{i}.pt'
    
    # Save the model
    torch.save(model.state_dict(), model_file1)
    
    model_file2 = f'./model_entire_{i}.pt'
    
    # Save the entire model
    torch.save(model, model_file2)


In [ ]:
def analyze_preds(labels, preds, confidences, confidence_score):
    # select samples where confidence >= confidence_score
    idx = confidences >= confidence_score
    labels = labels[idx]
    preds = preds[idx]
    
    acc = (preds == labels).sum() / labels.shape[0]
    bal_acc = balanced_accuracy_score(labels, preds)
    retained = idx.sum() / idx.shape[0]
    print(f"Retained {retained:.2f} of data")
    print(f"Accuracy: {acc:.2e}")
    print(f"Balanced Accuracy: {bal_acc:.2e}")

    cm = confusion_matrix(labels, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[classes[i] for i in range(len(classes)) if i in labels] )
    disp.plot()
    plt.show()    
    return bal_acc, retained / idx.shape[0]

In [ ]:
confidence_thresholds = [-3, -2, -1.5, -0.5, .0, .25, .5, .75, .9]
model_names = {
    0: "DenseNet",
    1: "ResNet",
    2: "GoogleNet"
}

# just if you need to reload from file
model_list = []
for i in range(len(model_names)):
    # Load the model from file model
    model = torch.load(f'./model_entire_multiclass_{i}.pt')
    
    # Load the state dictionary from the model_state file
    state_dict = torch.load(f'./model_state_multiclass_{i}.pt')
    
    # Load the state dictionary into the model
    model.load_state_dict(state_dict)
    model_list.append(model)

confidence_results = [[] for _ in model_list]
confidence_size = [[] for _ in model_list]
for i, model in enumerate(model_list):
    print("Testing " + model_names[i])
    labels, preds, confs = test_model(model, testset) 
    plt.hist(confs, bins=100)
    plt.gca().set(title='Confidence Histogram for ' + model_names[i], ylabel='Occurrences');
    plt.show()
    
    for confidence_threshold in confidence_thresholds:
        print(f"{model_names[i]} - Confidence {int(confidence_threshold * 100)}%")
        result, size = analyze_preds(labels, preds, confs, confidence_threshold)
        confidence_results[i].append(result)
        confidence_size[i].append(size)

In [ ]:
# Plotting confidence scores
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle('Results by confidence thresholds')

# plotting confidence
for i, result in enumerate(confidence_results):
    ax1.plot(confidence_thresholds, result, label=model_names[i])
# plotting data size
for i, size in enumerate(confidence_size):
    ax2.plot(confidence_thresholds, size, label=model_names[i])

# Set labels
ax1.set_ylabel('Balanced Accuracy')
# Display the legend
ax1.legend()

# Set labels
ax2.set_xlabel('Confidence scores')
ax2.set_ylabel('Percentage of retained dataset')
plt.show()

In [ ]:
def dataloader_to_numpy(dataloader):
    data_list = []
    label_list = []
    for data, label in tqdm(dataloader):
        # Reshape the data to 1D
        data = data.view(-1)
        data_list.append(data.numpy())
        label_list.append(label)
    return np.stack(data_list), np.asarray(label_list)

# Convert dataloaders to numpy arrays
print("Converting validation set")
valset.transform = transform_train
val_X, val_Y = dataloader_to_numpy(valset)
print("Converting train set")
train_X, train_Y = dataloader_to_numpy(trainset)
print("Converting test set")
test_X, test_Y = dataloader_to_numpy(testset)

# Concatenate training set and validation set
train_X = np.concatenate((train_X, val_X), axis=0)
train_Y = np.concatenate((train_Y, val_Y), axis=0)

In [ ]:
#make a dummy prediction, i.e Random Baseline
# Create a dummy classifier with a strategy
dummy_clf = DummyClassifier(strategy='uniform')

# Train the dummy classifier
dummy_clf.fit(train_X, train_Y)

# Make predictions on the test set
predictions = dummy_clf.predict(test_X)

# Evaluate the accuracy of the dummy classifier
accuracy = dummy_clf.score(test_X, test_Y)

print("Dummy Classifier Accuracy/ Random Baseline:", accuracy)

In [ ]:
# Predict using SVM
model_svc = SVC(decision_function_shape='ovr', C=100, kernel='rbf')
model_svc.fit(train_X, train_Y)
pred = model_svc.predict(test_X)
acc = accuracy_score(test_Y, pred)
print('Accuracy for SVM: ' + str(acc))
balanced_svm = balanced_accuracy_score(test_Y, pred)  # Calculate balanced accuracy
print("Balanced Accuracy:"+ str(balanced_svm))
# Confusion Matrix
cm = confusion_matrix(test_Y, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=classes )
disp.plot()
plt.figure(figsize=(8, 6))
plt.show()

In [ ]:
# Predict using MLP
model_mlp = mlp(solver='lbfgs', alpha=1e-5, random_state=5, max_iter=100000)
model_mlp.fit(train_X, train_Y)
pred = model_mlp.predict(test_X)
acc = accuracy_score(test_Y, pred)
print('Accuracy for MLP: ' + str(acc))
balanced_mlp = balanced_accuracy_score(test_Y, pred)  # Calculate balanced accuracy
print("Balanced Accuracy:"+ str(balanced_mlp))
# Confusion Matrix
cm = confusion_matrix(test_Y, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=classes )
disp.plot()
plt.figure(figsize=(8, 6))
plt.show()